In [ ]:
import os
import lightkurve as lk
import pandas as pd
import numpy as np
import dynesty
from background_fit import granulation_fit 
from matplotlib.pyplot import *
rcParams['font.size'] = 18

In [ ]:
download_dir = '/home/nielsemb/work/mounts/Bluebear_data/data'

workDir = '/home/nielsemb/work/repos/bfp9001'

prior_data = pd.read_csv(os.path.join(*[workDir, 'files', 'prior_data.csv']))

In [ ]:
updated_data = prior_data.copy()

new_keys = ['H1_power', 'H1_nu', 'H1_exp',
            'H2_power', 'H2_nu', 'H2_exp',
            'H3_power', 'H3_nu', 'H3_exp',
            'bkg_numax', 'bkg_envWidth', 'bkg_envHeight',
            'shot']

for key in new_keys:
    updated_data[key] = np.nan
    updated_data[key+'_err'] = np.nan

In [ ]:
idx = prior_data['ID'] == 'KIC6144777'

In [ ]:
prior_data[idx]

In [ ]:
figM, axM = subplots(figsize=(16,9))
fig3, ax3 = subplots(3,3, figsize=(9,9))
fig4, ax4 = subplots(4,4, figsize=(12,12))
figA, axA = subplots(13,13, figsize=(32,32))

cornerN = 5000

for i in prior_data.index[7655:7656]:

    ID = prior_data.loc[i, 'ID']
    print(ID)
    outputDir = os.path.join(*[workDir, 'results', ID])

    if not os.path.exists(outputDir):
        os.makedirs(outputDir)
        
    _numax = prior_data.loc[i, 'numax']
    numax = (10**_numax, 0.2*10**_numax)
 
    
    gfit = granulation_fit(ID, numax, download_dir)
    
    gfit.plotModel(figM, axM, outputDir=outputDir);
    axM.clear()
    
    sampler, samples = gfit.runDynesty()
 
    gfit.storeResults(i, updated_data, new_keys, samples, outputDir)
    
    gfit.plotModel(figM, axM, samples, outputDir=outputDir)
    axM.clear()
 
    fig, ax = gfit.makeCorner(fig3, samples, cornerN, labels=['hsig1', 'dhnu1', 'hexp1'], outputDir=outputDir);
    for ax in ax3.flatten():
        ax.clear()
 
    fig, ax = gfit.makeCorner(fig3, samples, cornerN, labels=['hsig2', 'dhnu2', 'hexp2'], outputDir=outputDir);
    for ax in ax3.flatten():
        ax.clear()
 
    fig, ax = gfit.makeCorner(fig3, samples, cornerN, labels=['hsig3', 'hnu3', 'hexp3'], outputDir=outputDir);
    for ax in ax3.flatten():
        ax.clear()
 
    fig, ax = gfit.makeCorner(fig4, samples, cornerN, labels=['numax', 'dwidth', 'height', 'white'], outputDir=outputDir);
    for ax in ax4.flatten():
        ax.clear()
 
    fig, ax = gfit.makeCorner(figA, samples, cornerN, labels=gfit.labels, outputDir=outputDir);
    for ax in axA.flatten():
        ax.clear()
        
updated_data.to_csv('prior_data_updated.csv', index=False)

In [ ]:
fileLoad = np.load('/home/nielsemb/work/repos/bfp9001/results/KIC4448777/KIC4448777_samples.npz')
samples = fileLoad['samples']

In [ ]:
samples.shape

In [ ]:
figM, axM = subplots(figsize=(16,9))
fig3, ax3 = subplots(3,3, figsize=(9,9))
fig4, ax4 = subplots(4,4, figsize=(12,12))
figA, axA = subplots(13,13, figsize=(32,32))

cornerN = 5000

ID = 'KIC8524425' #prior_data.loc[i, 'ID']

outputDir = os.path.join(*[workDir, 'results', ID])

if not os.path.exists(outputDir):
    os.makedirs(outputDir)

numax = (1100, 50)

gfit = granulation_fit(ID, numax, download_dir)

gfit.plotModel(figM, axM, outputDir=outputDir);
axM.clear()


sampler, samples = gfit.runDynesty()

gfit.storeResults(0, updated_data, new_keys, samples, outputDir)

gfit.plotModel(figM, axM, samples, outputDir=outputDir)
# axM.clear()

# fig, ax = gfit.makeCorner(fig3, samples, cornerN, labels=['hsig1', 'dhnu1', 'hexp1'], outputDir=outputDir);
# for ax in ax3.flatten():
#     ax.clear()

# fig, ax = gfit.makeCorner(fig3, samples, cornerN, labels=['hsig2', 'dhnu2', 'hexp2'], outputDir=outputDir);
# for ax in ax3.flatten():
#     ax.clear()

# fig, ax = gfit.makeCorner(fig3, samples, cornerN, labels=['hsig3', 'hnu3', 'hexp3'], outputDir=outputDir);
# for ax in ax3.flatten():
#     ax.clear()

# fig, ax = gfit.makeCorner(fig4, samples, cornerN, labels=['numax', 'dwidth', 'height', 'white'], outputDir=outputDir);
# for ax in ax4.flatten():
#     ax.clear()

# fig, ax = gfit.makeCorner(figA, samples, cornerN, labels=gfit.labels, outputDir=outputDir);
# for ax in axA.flatten():
#     ax.clear()
        
# updated_data.to_csv('prior_data_updated.csv', index=False)

In [ ]:
gfit.storeResults(0, updated_data, new_keys, samples, outputDir)

In [ ]:
figM, axM = subplots(figsize=(16,9))
fig3, ax3 = subplots(3,3, figsize=(9,9))
fig4, ax4 = subplots(4,4, figsize=(12,12))
figA, axA = subplots(13,13, figsize=(32,32))

gfit.storeResults(0, updated_data, new_keys, samples, outputDir)

gfit.plotModel(figM, axM, samples, outputDir=outputDir)


In [ ]:
y = -np.log(np.ones(len(gfit.f))) - np.log(np.random.uniform(size=len(gfit.f)))

In [ ]:
sum(y)

In [ ]:
figM, axM = subplots(figsize=(16,9))
axM.loglog(f[::10],p[::10], color = 'C5')
gfit.plotModel(figM, axM, samples, outputDir=outputDir)

In [ ]:
LCcol = lk.search_lightcurve(gfit.ID, author='Kepler', mission='Kepler', exptime=60).download_all(download_dir=gfit.download_dir)

wlen = int(1.5e6/60)-1
if wlen % 2 == 0:
    wlen += 1
lc = LCcol.stitch().normalize().remove_nans().remove_outliers().flatten(window_length=wlen)*1e6

t, d = lc.time.value, lc.flux.value

pg = lc.to_periodogram(normalization='psd', minimum_frequency=1.0)

f, p = pg.frequency.value, pg.power.value

tot_MS = np.nansum((d - np.nanmean(d))**2)/len(t)

normfactor = tot_MS/np.sum(p)

p *= normfactor

In [ ]:
from dataclasses import dataclass, astuple

@dataclass
class ThetaReg:
    '''
    Parameters for Reggae (asymptotic description of l = 1 modes)

    If θ is a numpy array, generate an instance as
    theta = ThetaReg(*θ)
    '''

    dPi0: float
    p_L: float
    p_D: float
    epsilon_g: float
    log_omega_rot: float
    d01: float
    alpha_g: float
    inclination: float

    

In [ ]:
ThetaReg.dims

ModuleNotFoundError: No module named 'LightkurveCacheAccess'